# Science User Case - Inspecting a Candidate List

Find available data for a list of objects. Here's the paper: https://ui.adsabs.harvard.edu//#abs/2016ApJ...817..109O/abstract

Table 1 has a list of super spirals.

## Write a query to get Table 1 from the Super Spirals paper in machine-readable form. I don't know how to do this!

## For each NED Name in the table, query NED to find the coordinates.

Hint: Use astroquery.ned!

In [1]:
from astroquery.ned import Ned
result_table = Ned.query_object("NGC 224")
#print(result_table) # an astropy.table.Table
result_table.colnames

['No.',
 'Object Name',
 'RA(deg)',
 'DEC(deg)',
 'Type',
 'Velocity',
 'Redshift',
 'Redshift Flag',
 'Magnitude and Filter',
 'Distance (arcmin)',
 'References',
 'Notes',
 'Photometry Points',
 'Positions',
 'Redshift Points',
 'Diameter Points',
 'Associations']

In [2]:
result_table['RA(deg)', 'DEC(deg)']

RA(deg),DEC(deg)
degrees,degrees
float64,float64
10.68479,41.269060000000003


## Grab the SDSS images for each super spiral

Hint: Use astroquery.sdss!

In [3]:
from astroquery.sdss import SDSS
from astropy import coordinates as coords
pos = coords.SkyCoord('0h8m05.63s +14d50m23.3s', frame='icrs')
xid = SDSS.query_region(pos, spectro=True)
print(xid)

      ra           dec             objid        ... run2d instrument
------------- ------------- ------------------- ... ----- ----------
2.02344596303 14.8398237521 1237652943176138868 ...    26       SDSS


/Users/desai/anaconda3/lib/python3.6/site-packages/astroquery/sdss/__init__.py:29: UserWarning: Experimental: SDSS has not yet been refactored to have its API match the rest of astroquery (but it's nearly there).
  warnings.warn("Experimental: SDSS has not yet been refactored to have its API "


In [4]:
sdss_images = SDSS.get_images(matches=xid, band='r')
len(sdss_images)
print(sdss_images)

[[<astropy.io.fits.hdu.image.PrimaryHDU object at 0x112a54588>, <astropy.io.fits.hdu.image.ImageHDU object at 0x112a5aef0>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x112a1b5c0>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x112a4feb8>]]


## Grab the WISE images for each super spiral

### Search the Registry for WISE image services.

In [5]:
# Find all WISE image services.
import sys
sys.path.append('workshop-dev-master')
from navo_utils.registry import Registry
from navo_utils.image import Image, ImageColumn
from navo_utils.spectra import Spectra, SpectraColumn
from navo_utils.cone import Cone
from navo_utils.tap import Tap
from navo_utils.utils import astropy_table_from_votable_response

wise_services = Registry.query(source='wise', service_type='image')


In [6]:
wise_services['short_name']

WISE 3-Band L3a
AllWISE L3a
WISE 3-Band L1b
NEOWISE-R L1b
WISE All-Sky L3A
WISE All-Sky L1b
WISE 2-Band L1b
WISE


In [7]:
search_list = wise_services[1]['access_url']
search_list

'https://irsa.ipac.caltech.edu/ibe/sia/wise/allwise/p3am_cdd?'

### Query the AllWISE image service for those that cover the Super Spirals.

In [8]:
# Specify the query coordinates.  
# The query expects a SkyCoord, or any string that can be parsed in to a SkyCoord.
from astropy.coordinates import SkyCoord

sc = SkyCoord(185.47873, 4.47365, unit="deg")

# Choose a search size. Zero means the image must contain the search point.
size = '0'  #diameter in degrees

# Perform the query on the WISE service.
result_list = Image.query(coords=sc, radius=size, service='https://irsa.ipac.caltech.edu/ibe/sia/wise/allwise/p3am_cdd?')

# The result is a list of tables, one for each coordinate in the query.
# In this case, we only input one coordinate, so there will only be one result 
# table in the list.
table = result_list[0]
table.show_in_notebook()

idx,sia_title,sia_url,sia_naxes,sia_fmt,sia_ra,sia_dec,sia_naxis,sia_crpix,sia_crval,sia_proj,sia_scale,sia_cd,sia_bp_id,sia_bp_ref,sia_bp_hi,sia_bp_lo,magzp,magzpunc,unc_url,cov_url,coadd_id
,,,,,deg,deg,,pix,deg,,deg / pix,deg / pix,,,,,,,,,
0,W3 Coadd 1853p045_ac51,https://irsa.ipac.caltech.edu/ibe/data/wise/allwise/p3am_cdd/18/1853/1853p045_ac51/1853p045_ac51-w3-int-3.fits,2,image/fits,185.31632500000001,4.5433329999999996,[4095 4095],[2048.0 2048.0],[185.316325 4.543333],SIN,[-0.0003819444391411 0.0003819444391411],[-0.0003819444391411 -0.0 -0.0 0.0003819444391411],W3,1.1559999999999999e-05,1.6269999999999998e-05,7.6000000000000001e-06,18.0,0.012,https://irsa.ipac.caltech.edu/ibe/data/wise/allwise/p3am_cdd/18/1853/1853p045_ac51/1853p045_ac51-w3-unc-3.fits.gz,https://irsa.ipac.caltech.edu/ibe/data/wise/allwise/p3am_cdd/18/1853/1853p045_ac51/1853p045_ac51-w3-cov-3.fits.gz,1853p045_ac51
1,W2 Coadd 1853p045_ac51,https://irsa.ipac.caltech.edu/ibe/data/wise/allwise/p3am_cdd/18/1853/1853p045_ac51/1853p045_ac51-w2-int-3.fits,2,image/fits,185.31632500000001,4.5433329999999996,[4095 4095],[2048.0 2048.0],[185.316325 4.543333],SIN,[-0.0003819444391411 0.0003819444391411],[-0.0003819444391411 -0.0 -0.0 0.0003819444391411],W2,4.6e-06,5.1900000000000003e-06,4.0199999999999996e-06,19.5,0.0070000000000000001,https://irsa.ipac.caltech.edu/ibe/data/wise/allwise/p3am_cdd/18/1853/1853p045_ac51/1853p045_ac51-w2-unc-3.fits.gz,https://irsa.ipac.caltech.edu/ibe/data/wise/allwise/p3am_cdd/18/1853/1853p045_ac51/1853p045_ac51-w2-cov-3.fits.gz,1853p045_ac51
2,W4 Coadd 1853p045_ac51,https://irsa.ipac.caltech.edu/ibe/data/wise/allwise/p3am_cdd/18/1853/1853p045_ac51/1853p045_ac51-w4-int-3.fits,2,image/fits,185.31632500000001,4.5433329999999996,[4095 4095],[2048.0 2048.0],[185.316325 4.543333],SIN,[-0.0003819444391411 0.0003819444391411],[-0.0003819444391411 -0.0 -0.0 0.0003819444391411],W4,2.209e-05,2.336e-05,1.984e-05,13.0,0.012,https://irsa.ipac.caltech.edu/ibe/data/wise/allwise/p3am_cdd/18/1853/1853p045_ac51/1853p045_ac51-w4-unc-3.fits.gz,https://irsa.ipac.caltech.edu/ibe/data/wise/allwise/p3am_cdd/18/1853/1853p045_ac51/1853p045_ac51-w4-cov-3.fits.gz,1853p045_ac51
3,W1 Coadd 1853p045_ac51,https://irsa.ipac.caltech.edu/ibe/data/wise/allwise/p3am_cdd/18/1853/1853p045_ac51/1853p045_ac51-w1-int-3.fits,2,image/fits,185.31632500000001,4.5433329999999996,[4095 4095],[2048.0 2048.0],[185.316325 4.543333],SIN,[-0.0003819444391411 0.0003819444391411],[-0.0003819444391411 -0.0 -0.0 0.0003819444391411],W1,3.3500000000000001e-06,3.7799999999999998e-06,3.1300000000000001e-06,20.5,0.0060000000000000001,https://irsa.ipac.caltech.edu/ibe/data/wise/allwise/p3am_cdd/18/1853/1853p045_ac51/1853p045_ac51-w1-unc-3.fits.gz,https://irsa.ipac.caltech.edu/ibe/data/wise/allwise/p3am_cdd/18/1853/1853p045_ac51/1853p045_ac51-w1-cov-3.fits.gz,1853p045_ac51


In [9]:
import workshop_utils.utils
url = 'https://irsa.ipac.caltech.edu/ibe/data/wise/allwise/p3am_cdd/18/1853/1853p045_ac51/1853p045_ac51-w1-int-3.fits'
out_file_path = workshop_utils.utils.download_file(url, directory='.')

## Grab the HST images for each super spiral

## Plot the SDSS and WISE images next to each other